3-1. 네이버 책 검색 API 호출하기 (필수)
https://developers.naver.com/docs/serviceapi/search/book/book.md#%EC%B1%85

검색어(query)는  함수의 인자로 받아서 동적으로 처리 되어야 합니다. 
최대한 코드가 중복되지 않도록 공통으로 처리해야 하는 부분은 함수로 작성해서 재사용 하는 방식으로 코드를 작성해 주세요.

def search_books(query):   
    # query string 문자열을 dict 선언
    payload = {
    'query':  query, #'파이썬',
    'display': 50,
    'sort': 'sim'
    }

   url = 'https://openapi.naver.com/v1/search/book.json'
   
   # requests get(url, params, headers) 요청 
   res = requests.get(url, params=payload, headers=headers)
   # json() 함수로 응답 결과 가져오기
   items_data = res.json()['items'] 

1. 질문 :  검색어로  찾은  책 목록을 json 파일로 저장하기
   data/books.json 파일로 저장해 주세요.


In [22]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [33]:
import os #폴더 만들기 환경변수읽기 등 파일관련할때 쓰는 기능
import json #제이슨파일 읽고 쓰기
import requests # api요청 할때씀
import pandas as pd # 데이터 표형태로 쓰기위해서
from dotenv import load_dotenv #환경변수 env 파일 읽어오려고 사용
from IPython.display import display


load_dotenv() # 환경변수 읽어오기

CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET") #값 받아왔음

HEADERS = { #헤더에 id key값 넣기
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret": CLIENT_SECRET
} 

JSON_PATH = "data/books.json"  #저장할 공간 경로

def ensure_data_dir(): #data폴더가 없으면 자동으로 만드는 함수 
    os.makedirs("data", exist_ok=True)

def to_int_safe(x): # 논타입을 정수로 반환하고 실패하면 0반환한다는데 왜 쓴지는 잘 모르겠음
    try:
        return int(x)
    except:
        return 0


In [34]:
# 1. 질문 :  검색어로  찾은  책 목록을 json 파일로 저장하기
#    data/books.json 파일로 저장해 주세요.

def search_books(query): 
    payload = { #쿼리는 입력받아서 검색할 책이름이고 50개까지만 관련도순으로 정렬한다는 뜻
        "query": query,
        "display": 50,
        "sort": "sim"
    }
    url = "https://openapi.naver.com/v1/search/book.json"

    res = requests.get(url, params=payload, headers=HEADERS)
    res.raise_for_status()  # 200 아니면 예외 발생 네이버 책 검색 API 호출 (에러가 나면 멈춤)

    items = res.json().get("items", []) #응답제이슨에서 item만 추출한다 책목록 

    # 1. JSON 저장
    ensure_data_dir()
    with open(JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(items, f, ensure_ascii=False, indent=2)

    # 2. DataFrame 생성해서 반환
    df = pd.DataFrame(items)
    return df


In [35]:
#2. books.json 파일을 Pandas DataFrame로 저장하기
def load_books_df_from_json():
    with open(JSON_PATH, "r", encoding="utf-8") as f:
        items = json.load(f)
    return pd.DataFrame(items)

In [36]:
# 3. 질문 :  검색어로  찾은  책 목록 출력하기
# 출력결과 : 

def get_all_books(df):
    cols = ["title", "author", "discount", "publisher", "pubdate", "isbn"]
    exist_cols = [c for c in cols if c in df.columns]
    return df[exist_cols].reset_index(drop=True)


In [40]:
# 4. 질문 :  검색어로  찾은  책 목록 중에서 가격이 2만원 이상인 책만 출력하기

# title,author,discount,publisher,pubdate 컬럼만 출력
# 가격은 descending (내림차순), index 초기화

def get_books_over_20000(df):
    if "discount" not in df.columns:
        return pd.DataFrame()

    tmp = df.copy()
    tmp["discount_int"] = tmp["discount"].apply(to_int_safe)

    # 💡 컬럼 이름 미리 rename할 거니까, discount로 써야 함
    cols = ["title", "author", "discount", "publisher", "pubdate"]

    # 필터링
    filtered = (
        tmp[tmp["discount_int"] >= 20000]
        .sort_values(by="discount_int", ascending=False)
        .reset_index(drop=True)
        .rename(columns={"discount_int": "discount"})  # 미리 이름 바꿈
    )

    exist_cols = [c for c in cols if c in filtered.columns]
    return filtered[exist_cols]



In [41]:
def get_books_infinitybooks(df):
    if "publisher" not in df.columns:
        return pd.DataFrame()

    tmp = df[df["publisher"] == "인피니티북스"].reset_index(drop=True)
    drop_cols = [c for c in ["image", "description"] if c in tmp.columns]
    tmp = tmp.drop(columns=drop_cols, errors="ignore")

    return tmp


In [44]:
if __name__ == "__main__":
    query = input("검색어를 입력하세요: ").strip()

    try:
        df_books = search_books(query)

        df1 = get_all_books(df_books)
        df2 = get_books_over_20000(df_books)
        df3 = get_books_infinitybooks(df_books)

    except requests.exceptions.HTTPError as e:
        print("HTTP 에러:", e)
    except Exception as e:
        print("예외 발생:", e)



In [ ]:
display(df1) #책목록

,title,author,discount,publisher,pubdate,isbn
0,처음이야? 파이썬 기초 (동영상 강의로 배우는 292개 코드 따라하기(핵심노트+오픈...,윤영빈^오환^이용희,18000,영진닷컴,20250620,9788931478006
1,모두의 인공지능 with 파이썬 (누구나 쉽게 시작하는 AI 기초 프로그래밍),이영호,27000,길벗,20250725,9791140714995
2,혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서),윤인성,19800,한빛미디어,20220601,9791162245651
3,Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서),박응용,19800,이지스퍼블리싱,20230615,9791163034735
4,파이썬,홍의경,30400,생능출판,20220309,9788970506784
5,파이썬,천인국,10800,인피니티북스,20170830,9791185578330
6,CODING BASICS PYTHON (파이썬),김상민^장성식^김일태,18000,렉스미디어닷넷,20220210,9788959604210
7,파이썬 (제2판),염기원^오지영,16200,북두,20220120,9791166750991
8,파이썬 (컴퓨팅 사고력을 키우는 SW 교육),우재남,20460,한빛아카데미,20160720,9791156642701
9,파이썬 (비전공자를위한 파이썬 기초 입문서),김명호,16200,에듀웨이,20190525,9791186179369


In [ ]:
display(df2)#2만원이상

In [ ]:
display(df3) #인피니티북스